In [1]:
import os
import cv2
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import skimage
import imantics
import json
import shutil

# Root directory of the project
ROOT_DIR = 'Mask_RCNN-master 3'

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import Cell

import imantics
from IPython.display import clear_output

Using TensorFlow backend.


## Helper functions

In [2]:
# Function taken from utils.dataset
def load_image(image_path):
    """Load the specified image and return a [H,W,3] Numpy array.
    """
    # Load image
    image = skimage.io.imread(image_path)
    # If grayscale. Convert to RGB for consistency.
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
    # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    return image
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    ,
            "mrcnn_bbox", "mrcnn_mask"
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Begin

In [ ]:
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import skimage
iteration = 0 #modify to resume at iteration
current_count = 30
iteration_folder = os.path.abspath("../iterations/") + "/"

dataset = Cell.HSYAADataset()
dataset.load_data("dataset/", "train")
dataset.prepare()

batch_size = 10 #Small batch sizes early, large batch sizes later
validation_percent = 50 # Typically 10% but larger for very small datasets

time_per_trap = 391
beacon = 8
traps_folder = "/home/ubuntu/github/HSYAA_GT/IM_with_GT/BC8_good_gray/Trap010/"
output_folder = "/Users/trevorpeyton/Documents/github/HSYAA_GT/IM_with_GT/predicted/"
image_format = "rf02032017_TL1444143_BC%d_Tp%.3dTm%.3d.jpg"
traps = [o[-3:] for o in os.listdir(traps_folder) if os.path.isdir(os.path.join(traps_folder,o))]

generate_batch = False

annotation_train_json = {}
annotation_val_json = {}

while(input("Press s to begin next iteration. Press anything else to stop.") == 's'):
    clear_output(wait=True)
    
    print("Creating iteration...")
    
    if iteration > 0:
        if not os.path.exists(iteration_folder + "iteration%.3d" % (iteration - 1)):
            print("Error: the last iteration (%.3d) doesn't exist." % (iteration - 1))
            print("Stopping...")
            break
    
    current_iteration_folder = iteration_folder + "iteration%.3d/" % iteration
    
    if not os.path.exists(current_iteration_folder):
        os.mkdir(current_iteration_folder)
        os.mkdir(current_iteration_folder + "/dataset/")
        os.mkdir(current_iteration_folder + "/dataset/train/")
        os.mkdir(current_iteration_folder + "/dataset/val/")
    elif os.path.exists(current_iteration_folder + "final_iteration_model.h5"):
        if input("Iteration already exists would you like to skip to the next? y/n") == "y":
            iteration += 1
        
    print("Loading model...")
    
    training_folder = os.path.abspath(current_iteration_folder + "dataset/train/") + "/"
    val_folder = os.path.abspath(current_iteration_folder + "dataset/val/") + "/"
    
    model_dir = model_file = model_path = ""
    if iteration == 0:
        model_dir = "../logs/cell20200529T1324/"
        model_file = "mask_rcnn_cell_0050.h5"
        model_path = os.path.abspath(model_dir + model_file)
    else:
        model_dir = iteration_folder + "iteration%.3d/" % (iteration - 1)
        model_file = "final_iteration_model.h5"
        model_path = os.path.abspath(model_dir + model_file)
        
    predict_model = modellib.MaskRCNN(mode="inference", config=Cell.config, model_dir=model_dir)
    predict_model.load_weights(model_path, by_name=True)
    training_model = modellib.MaskRCNN(mode="training", config=Cell.config, model_dir=model_dir)
    training_model.load_weights(model_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
    
    print("Generating batch...")
    
    debug = {}
        
    if generate_batch:
        batch_size = int(input("New batch size:"))            
        validation_percent = int(input("New validation percent:"))
    
        validation_list = random.sample(range(current_count, current_count + batch_size), int(batch_size * (validation_percent / 100)))
        for n in range(current_count, current_count + batch_size):
            validation = n in validation_list
            trap = int(n / time_per_trap)
            time = (n % time_per_trap) + 1
            current_count = n

            filename = image_format % (beacon, int(traps[trap]), time)
            filepath = os.path.join(traps_folder, "Trap" + traps[trap], filename)

            print(f"Trap: {traps[trap]}, Time: {time}")

            #Copy
            img = cv2.imread(filepath, 0)
            img = cv2.resize(img, (256, 256))
            if validation:
                cv2.imwrite(current_iteration_folder + "dataset/val/" + filename, img)
            else:
                cv2.imwrite(current_iteration_folder + "dataset/train/" + filename, img)

            # Run object detection
            results = predict_model.detect([cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)], verbose=1)

            # Display results
            #ax = get_ax(1)
            r = results[0]

            a = visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                       dataset.class_names, r['scores'], ax=ax,
                                       title="Predictions", path=os.path.join(new_output_folder, filename))

            clear_output(wait=True)

            regions = []

            for n in range(r['masks'].shape[2]):
                poly = imantics.Polygons.from_mask(imantics.Mask(r['masks'][:, :, n]))
                points = [y for x in poly.segmentation for y in x]
                try:
                    regions.append({
                            "shape_attributes": {
                                "name": "polygon",
                                "all_points_x": points[::2],
                                "all_points_y": points[1::2]
                            },
                            "region_attributes": {
                                "name": dataset.class_names[r['class_ids'][n]]
                            }
                        })
                except Exception as e:
                    mask = r['masks'][:, :, n]
                    print(points)
                    
                    if hasattr(e, 'message'):
                        print(e.message)
                    else:
                        print(e)
                        
                    _ = input("")

            file_annotations = {
                    "filename": filename,
                    "regions": regions,
                    "file_attributes": {}
                }

            if validation:
                annotation_val_json[filename] = file_annotations
            else:
                annotation_train_json[filename] = file_annotations

        via_project = {
            "_via_settings": {
                "ui": {
                    "annotation_editor_height": 25,
                    "annotation_editor_fontsize": 0.8,
                    "leftsidebar_width": 18,
                    "image_grid": {
                        "img_height": 80,
                        "rshape_fill": "none",
                        "rshape_fill_opacity": 0.3,
                        "rshape_stroke": "yellow",
                        "rshape_stroke_width": 2,
                        "show_region_shape": True,
                        "show_image_policy": "all"
                    },
                    "image": {
                        "region_label": "__via_region_id__",
                        "region_color": "__via_default_region_color__",
                        "region_label_font": "10px Sans",
                        "on_image_annotation_editor_placement": "NEAR_REGION"
                    }
                },
                "core": {
                    "buffer_size": "18",
                    "filepath": {},
                    "default_filepath": ""
                },
                "project": {
                    "name": "via_project_with"
                }
            },
            "_via_img_metadata": {},
            "_via_attributes": {
                "region": {
                    "name": {
                        "type": "text"
                    }
                },
                "file": {}
                
            }
        }

        via_project["_via_img_metadata"] = annotation_train_json
        via_project["_via_settings"]["core"]["default_filepath"] = training_folder
        print(current_iteration_folder)
        with open(current_iteration_folder + 'dataset/train/via_project.json', 'w') as outfile:
            json.dump(via_project, outfile)

        via_project["_via_img_metadata"] = annotation_val_json
        via_project["_via_settings"]["core"]["default_filepath"] = val_folder
        with open(current_iteration_folder + 'dataset/val/via_project.json', 'w') as outfile:
            json.dump(via_project, outfile)
            
    generate_batch = True
    
    clear_output(wait=True)
    
    print("Images moved and predictions saved...")
    
    print("Open VIA and update any missing annotations for BOTH training and validation.")
    
    input("Type anything to continue once you have updated all the missing annotations AND exported as json and overriden the via_export_json.json")
    
    input("Buffer input just for safety")
    
    clear_output(wait=True)
    
    epochs = int(input("Epochs?"))
    
    print("Training based on new annotations")
    
    Cell.train(training_model, training_model.epoch + epochs, os.path.abspath(current_iteration_folder + "dataset/") + "/")
    
    training_model.keras_model.save_weights(current_iteration_folder + "final_iteration_model.h5")
    
    print("Saved")
    
    iteration += 1

Images moved and predictions saved...
Open VIA and update any missing annotations for BOTH training and validation.
